In [4]:
import torch as th
import numpy as np
import pandas as pd


In [30]:
a_vec = th.tensor([3, 2])
b_vec = th.tensor([1, 1])

print(a_vec - th.min(a_vec // b_vec) * b_vec)

tensor([1, 0])


In [250]:
class Module:
    def __init__(self, mod_idx, prod_idx, prod_vec, max_capacity):

        self.prod_vec = prod_vec
        self.corrected_prod_vec = th.where(self.prod_vec > 0, self.prod_vec, th.iinfo(th.int32).max)

        if th.sum(prod_vec) > 0:
            self.origin_mod = False
        else:
            self.origin_mod = True

        self.mod_idx = mod_idx
        self.prod_idx = prod_idx
        self.max_capacity = max_capacity

    def process(self, input_tensor):

        output_tensor = input_tensor
        if self.origin_mod:
            output_tensor[1, self.mod_idx, self.prod_idx] += self.max_capacity
        else:
            current_vec = output_tensor[0, self.mod_idx, :]
            current_inventory = th.min(th.where(self.prod_vec > 0, current_vec // self.corrected_prod_vec, th.iinfo(th.int32).max))
            num_produced = min(current_inventory, self.max_capacity)
            output_tensor[0, self.mod_idx, :] -= self.prod_vec * num_produced
            output_tensor[1, self.mod_idx, self.prod_idx] += num_produced

        return output_tensor

class Transition:
    def __init__(self, input_mod_idx, output_mod_idx, prod_idx, max_capacity):
            self.input_mod_idx = input_mod_idx
            self.output_mod_idx = output_mod_idx
            self.prod_idx = prod_idx
            self.max_capacity = max_capacity

    def process(self, input_tensor):
        output_tensor = input_tensor
        current_capacity = output_tensor[1, self.input_mod_idx, self.prod_idx]
        num_transport = min(self.max_capacity, current_capacity)
        output_tensor[0, self.output_mod_idx, self.prod_idx] += num_transport
        output_tensor[1, self.input_mod_idx, self.prod_idx] -= num_transport


        return output_tensor



In [251]:

prod_matrix = th.tensor([[0, 0, 0],
                         [1, 0, 1],
                         [0, 0, 0]], dtype=th.int32)

mod_a = Module(0, 0, prod_matrix[0], 10)
mod_b = Module(1, 1, prod_matrix[1], 10)
mod_c = Module(2, 2, prod_matrix[2], 11)

tra_ab = Transition(0, 1, 0, 10)
tra_cb = Transition(2, 1, 2, 11)

sc = [mod_a, tra_ab, mod_c, tra_cb, mod_b]

prod_tensor = th.zeros(2, 3, 3, dtype=th.int32)

for i in range(10000):
    for mod in sc:
        prod_tensor = mod.process(prod_tensor)

print(prod_tensor)

tensor([[[     0,      0,      0],
         [     0,      0,  10000],
         [     0,      0,      0]],

        [[     0,      0,      0],
         [     0, 100000,      0],
         [     0,      0,      0]]], dtype=torch.int32)
